# Linuxの操作メモ
忘れがちだけど時たま使う操作や苦手な操作をメモしておく。

## 任意のディレクトリ配下の容量を表示
duコマンドに-hオプションをつけることでSI単位で容量を表示する。

```
 du -h 容量を調べたいディレクトリ
```

## 任意のディレクトリ配下のうち、大きいサイズのディレクトリを調べる
sort -rnは-n:numericに比較、-r: 降順に並べるの意味。

```
du . | sort -rn | head -10
```

## 連番で空ファイルを作成する
```
for i in $(seq 3); do touch file_$i.txt ;done
```

## ディレクトリのみ表示する
```
ls -d */
```

In [4]:
! ls

AA101  AA102  AA201  file_1.txt  file_2.txt  file_3.txt  practice_memo.ipynb


In [1]:
! ls -d */

AA101/	AA102/	AA201/


## カレントディレクトリの全ディレクトリに空ファイルを作成する

In [5]:
! cat touch_file.sh

#!/bin/sh

dir_list=$(ls -d */ | cut -c 1-5)

for dir in $dir_list;
do
	for i in $(seq 3);
	do
		touch ./$dir/file_$i.txt;
	done;
done;



## ファイルを1行ずつ処理する

1行ずつファイルの中身をechoで標準出力する例。
```
 cat ファイル名 | while read line; do echo $line; done
```

## ファイルの1行目に任意の文字列を入れる

下記のように書けば良い。  
1iで1行目に追記するの意。2iとすれば２行目に挿入される。    

```
sed -i '1i 入れたい文字列' 対象ファイル名
```

## xargsで複数ファイルをまとめて操作

### ・例1：カレントディレクトリにある拡張子が.txtのファイルをtempディレクトリへコピー  

```
ls *.txt | xargs -I{} cp {} temp
```
-I{}で{}の位置にパイプからの引数を入れるという操作になる。

### ・例2： findの結果にヒットしたファイルを削除

```
find temp -regex ".*file_[12].*" | xargs -I{} rm {}
```
上記ではtemp配下のfile_1.txtまたはfile_2.txtを削除している。

### ・例3： 削除したいファイルのリストを読み込んで削除
```
cat rm_files_list.txt | xargs -I{} rm {}
```

### ※補足
-pオプションをつけることで、ドライランで実際に実行されるコマンドを確認できる

```
find temp -regex ".*file_[12].*" | xargs -I{} -p rm {}
```

## 2つのディレクトリを同期する
rsyncを使うことで、ディレクトリ内の各ファイルのタイムスタンプ、サイズを比較して  
変更があった分だけ同期できる。

```
rsync -av 同期元/ 同期先/
```
-aはサブディレクトリも含めて再帰的に処理するオプション。  
パーミッションやファイルのタイムスタンプも保持される。  
-vはverbose。同期元と同期先の最後に/をつけないと、  
同期元のフォルダ自体と等しいか否かを判定してしまい、まるごと同期されてしまう。

## 既にファイルがあったら処理をしない
例として、ディレクトリのリストがあり、これらのディレクトリをtar -cvzfでアーカイブ化＆圧縮したいが、  
既に処理済みのものは処理をスキップしたい場合を考える。  
以下のようにif文でtarファイルの存在有無を確認すればよい。

In [9]:
! cat practice-2_Hanling_list_of_files/execute_tar.sh

#!/bin/bash

target_dirs=$(ls -d */)

for dir in $target_dirs;
do
	base_dir=$(basename $dir) 
	if [ -e ${base_dir}.tar ]; then
		echo "${base_dir}.tar already exists. skip tar."
	else
		tar -czvf ${base_dir}.tar $dir
	fi
done


## プロセスを特定の論理CPU上で動かす
下記を実行すると、<コマンド>引数で指定したコマンドを-c <論理CPU番号>引数で指定した  
CPU上で実行できる。特定のコアにバインドしてプログラムを動かしたいときに使える。
```
taskset -c <論理CPU番号> <コマンド>
```

## コマンドをバックグランドで実行する
&を末尾につけて実行すればよい。  
任意のコマンドを実行しつつターミナルで作業したいときに使える。  
```
<コマンド> &
```

## プロセスが発行するシステムコールを確認
プロセスがどんなシステムコールを発行するかはstraceコマンドで確認できる。
```
strace -o 出力先 プログラムのパス
```
straceの出力は１行１行が１システムコールに対応している。  
また、-Tオプションをつけることで各システムコールにかかった時間、  
-ttでシステムコールの発行時刻をマイクロ秒単位で確認可能。

## システムに存在する全プロセスを列挙する
```
ps aux
```
※auxの意味：
* a: 端末操作のプロセス(自分 + 他ユーザー)を表示する
* u: 各プロセスの実行ユーザーやCPU, Mem(メモリ)等の情報も表示する
* x: 端末操作のないプロセス(daemon等)も表示する

さらに f をつけて実行するとプロセス間の親子関係をtreeの形で表示できる。